In [38]:
from pyspark.sql import SparkSession
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [39]:
spark = SparkSession.builder.appName("Spark-Job-1").master("local[*]").getOrCreate()

In [40]:
data = spark.read.format("csv").option("path", "file:///media/zmwaris/mydrive/digivate/train_V2.csv") \
.option("header", True).option("inferSchema", True).load()

In [41]:
data.limit(10).toPandas().head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [42]:
data.summary().show(truncate=False)

+-------+-----------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+---------+------------------+------------------+-----------------+-------------------+------------------+---------------------+------------------+-------------------+-------------------+------------------+------------------+-----------------+------------------+
|summary|Id               |groupId           |matchId           |assists            |boosts            |damageDealt       |DBNOs             |headshotKills      |heals             |killPlace        |killPoints        |kills             |killStreaks       |longestKill       |matchDuration     |matchType|maxPlace          |numGroups         |rankPoints       |revives            |rideDistance      |roadKills            |swimDistance      |team

In [43]:
data.printSchema()

root
 |-- Id: string (nullable = true)
 |-- groupId: string (nullable = true)
 |-- matchId: string (nullable = true)
 |-- assists: integer (nullable = true)
 |-- boosts: integer (nullable = true)
 |-- damageDealt: double (nullable = true)
 |-- DBNOs: integer (nullable = true)
 |-- headshotKills: integer (nullable = true)
 |-- heals: integer (nullable = true)
 |-- killPlace: integer (nullable = true)
 |-- killPoints: integer (nullable = true)
 |-- kills: integer (nullable = true)
 |-- killStreaks: integer (nullable = true)
 |-- longestKill: double (nullable = true)
 |-- matchDuration: integer (nullable = true)
 |-- matchType: string (nullable = true)
 |-- maxPlace: integer (nullable = true)
 |-- numGroups: integer (nullable = true)
 |-- rankPoints: integer (nullable = true)
 |-- revives: integer (nullable = true)
 |-- rideDistance: double (nullable = true)
 |-- roadKills: integer (nullable = true)
 |-- swimDistance: double (nullable = true)
 |-- teamKills: integer (nullable = true)
 |--

In [44]:
data.limit(10).toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 29 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               10 non-null     object 
 1   groupId          10 non-null     object 
 2   matchId          10 non-null     object 
 3   assists          10 non-null     int32  
 4   boosts           10 non-null     int32  
 5   damageDealt      10 non-null     float64
 6   DBNOs            10 non-null     int32  
 7   headshotKills    10 non-null     int32  
 8   heals            10 non-null     int32  
 9   killPlace        10 non-null     int32  
 10  killPoints       10 non-null     int32  
 11  kills            10 non-null     int32  
 12  killStreaks      10 non-null     int32  
 13  longestKill      10 non-null     float64
 14  matchDuration    10 non-null     int32  
 15  matchType        10 non-null     object 
 16  maxPlace         10 non-null     int32  
 17  numGroups        10

In [67]:
from pyspark.sql.functions import mean, max
from pyspark.sql import DataFrameStatFunctions as df
maxk = data.select(max(data.kills))
avg = data.select(mean(data.kills))
q = df(data).approxQuantile("kills", [0.99], 0)
val = maxk.collect()[0]['max(kills)']
val2 = avg.collect()[0]['avg(kills)']
print("Max kills", val)
print("Average kills", val2)
q.show()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/media/zmwaris/mydrive/apps/Apache_Spark/spark-3.4.0-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/media/zmwaris/mydrive/apps/Apache_Spark/spark-3.4.0-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/media/zmwaris/mydrive/apps/Apache_Spark/spark-3.4.0-bin-hadoop3/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


Py4JError: An error occurred while calling o342.apply

TypeError: 'Column' object is not callable